In [20]:
import json
import pandas as pd 

In [1]:
import selenium

import requests
from selenium import webdriver

import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

driver = webdriver.Chrome("C:\\Users\\brend\\chromedriver\\chromedriver_win32\\chromedriver.exe")
driver.get('https://opensea.io/rankings?sortBy=total_volume')
driver.implicitly_wait(1)
WebDriverWait(driver,5)
lines = driver.find_elements(By.CLASS_NAME, value = 'styles__StyledLink-sc-l6elh8-0')
url_lst = [a.get_attribute('href') for a in lines]
for i in range(len(url_lst)):
    print(url_lst[i])

print(driver.title)

driver.quit()

url_list = url_lst[2:]
split = [i.split("/")for i in url_list]
collection_names = [i[-1] for i in split]




In [17]:
for b in collection_names:
    print( "https://api.opensea.io/api/v1/collection/{}/stats".format(b))


https://api.opensea.io/api/v1/collection/art-blocks/stats
https://api.opensea.io/api/v1/collection/meebits/stats
https://api.opensea.io/api/v1/collection/cryptokitties/stats
https://api.opensea.io/api/v1/collection/clonex/stats
https://api.opensea.io/api/v1/collection/cool-cats-nft/stats
https://api.opensea.io/api/v1/collection/lootproject/stats
https://api.opensea.io/api/v1/collection/decentraland-wearables/stats
https://api.opensea.io/api/v1/collection/sandbox/stats
https://api.opensea.io/api/v1/collection/rarible/stats
https://api.opensea.io/api/v1/collection/mutant-ape-yacht-club/stats
https://api.opensea.io/api/v1/collection/boredapeyachtclub/stats
https://api.opensea.io/api/v1/collection/decentraland/stats
https://api.opensea.io/api/v1/collection/cryptopunks/stats


In [33]:

for b in collection_names:
    url = f"https://api.opensea.io/api/v1/collection/{b}/stats"

    headers = {"Accept": "application/json"}
    response = requests.request("GET", url, headers=headers)
    print(response.status_code)
    nft_dict = response.json()
    
    nft_dict[str(b)] = nft_dict['stats']


200
200
200
200
200
200
200
200
200
200
200
200
200


In [34]:
nft_dict

{'stats': {'one_day_volume': 7859.319999999996,
  'one_day_change': 1.8853188443041218,
  'one_day_sales': 118.0,
  'one_day_average_price': 66.60440677966098,
  'seven_day_volume': 18275.71660000002,
  'seven_day_change': 3.661004972733878,
  'seven_day_sales': 263.0,
  'seven_day_average_price': 69.4894167300381,
  'thirty_day_volume': 36444.37060000002,
  'thirty_day_change': -0.06644267222459975,
  'thirty_day_sales': 511.0,
  'thirty_day_average_price': 71.31970763209397,
  'total_volume': 802151.3228492871,
  'total_sales': 18828.0,
  'total_supply': 9999.0,
  'count': 9999.0,
  'num_owners': 3386,
  'average_price': 42.60417053586611,
  'num_reports': 5,
  'market_cap': 694824.677883651,
  'floor_price': None},
 'cryptopunks': {'one_day_volume': 7859.319999999996,
  'one_day_change': 1.8853188443041218,
  'one_day_sales': 118.0,
  'one_day_average_price': 66.60440677966098,
  'seven_day_volume': 18275.71660000002,
  'seven_day_change': 3.661004972733878,
  'seven_day_sales': 263

In [29]:
stats_df= pd.DataFrame(nft_dict).T
stats_df

,one_day_volume,one_day_change,one_day_sales,one_day_average_price,seven_day_volume,seven_day_change,seven_day_sales,seven_day_average_price,thirty_day_volume,thirty_day_change,...,thirty_day_average_price,total_volume,total_sales,total_supply,count,num_owners,average_price,num_reports,market_cap,floor_price
stats,8760.18,6.051582,130.0,67.386,17636.2566,3.854445,254.0,69.434081,35516.9406,-0.094791,...,71.319158,801223.892849,18815.0,9999.0,9999.0,3384.0,42.584315,5.0,694271.376943,NaN
cryptopunks,8760.18,6.051582,130.0,67.386,17636.2566,3.854445,254.0,69.434081,35516.9406,-0.094791,...,71.319158,801223.892849,18815.0,9999.0,9999.0,3384.0,42.584315,5.0,694271.376943,NaN


In [ ]:
for i in collection_names:
    url = "https://api.opensea.io/api/v1/collection/{}".format(x)

    response = requests.request("GET", url)

    col_dict = response.json()

  
    col2_dict['i'] = col_dict['collection']




collection_df = pd.DataFrame(col2_dict).T

In [64]:

pd.DataFrame(import_api(names)).T

,one_day_volume,one_day_change,one_day_sales,one_day_average_price,seven_day_volume,seven_day_change,seven_day_sales,seven_day_average_price,thirty_day_volume,thirty_day_change,...,thirty_day_average_price,total_volume,total_sales,total_supply,count,num_owners,average_price,num_reports,market_cap,floor_price
sandbox,599.637114,-0.020624,158.0,3.795172,4598.414463,-0.379595,1183.0,3.887079,25103.181476,-0.115966,...,3.983367,139896.378133,77123.0,104271.0,104271.0,18836.0,1.813938,8.0,405309.614978,3.479000
cryptopunks,908.970000,-0.676867,13.0,69.920769,9277.206600,1.796191,125.0,74.217653,27647.540600,-0.280335,...,73.335651,791961.382849,18676.0,9999.0,9999.0,3373.0,42.405300,5.0,742102.310347,NaN
boredapeyachtclub,2295.496900,0.299242,25.0,91.819876,12794.140800,0.007355,136.0,94.074565,68824.012997,1.219095,...,83.931723,339393.320702,24133.0,10000.0,10000.0,6182.0,14.063453,18.0,940745.647059,88.000000
decentraland,391.432836,1.214873,59.0,6.634455,2400.892218,0.385542,306.0,7.846053,7862.071942,-0.175130,...,6.210167,231750.294626,17243.0,97240.0,97240.0,6530.0,13.440254,1.0,762950.193780,4.168632
art-blocks,379.760871,9.052222,211.0,1.794367,1002.779283,-0.295394,451.0,2.219965,8624.076799,-0.131652,...,2.942587,221364.008397,48769.0,48578.0,48578.0,10308.0,4.539044,0.0,107841.470859,NaN


In [67]:
dataframe1

,one_day_volume,one_day_change,one_day_sales,one_day_average_price,seven_day_volume,seven_day_change,seven_day_sales,seven_day_average_price,thirty_day_volume,thirty_day_change,...,thirty_day_average_price,total_volume,total_sales,total_supply,count,num_owners,average_price,num_reports,market_cap,floor_price
0,2138.41,0.994804,22.0,97.200455,12590.829,-0.207046,137.0,91.903861,65359.670897,1.288252,...,81.293123,331082.623309,24045.0,10000.0,10000.0,6169,13.769292,12,919038.613139,84.5
1,2138.41,0.994804,22.0,97.200455,12590.829,-0.207046,137.0,91.903861,65359.670897,1.288252,...,81.293123,331082.623309,24045.0,10000.0,10000.0,6169,13.769292,12,919038.613139,84.5


In [ ]:
project should have begining and end - come to a conclusion and answer the question asked 

In [ ]:
needs to be concise - 10 min presentation 

In [ ]:
BEautiful ai -- presentation maker 

In [29]:
dff

,banner_image_url,chat_url,created_date,default_to_fiat,description,dev_buyer_fee_basis_points,dev_seller_fee_basis_points,discord_url,display_data,editors,...,primary_asset_contracts,require_email,safelist_request_status,short_description,slug,stats,telegram_url,traits,twitter_username,wiki_url
collection,https://lh3.googleusercontent.com/svc_rQkHVGf3...,None,2021-10-17T13:00:47.419945,False,A community-driven collectibles project featur...,0,500,https://discord.gg/doodles,{'card_display_style': 'contain'},"[0x0239769a1adf4def9f07da824b80b9c4fcb59593, 0...",...,[{'address': '0x8a90cab2b38dba80c64b7734e58ee1...,False,verified,None,doodles-official,"{'one_day_volume': 441.15000000000003, 'one_da...",None,"{'background': {'blue': 726, 'blue space': 1, ...",doodles,None
